In [47]:
import numpy as np
import pandas as pd

In [48]:
# Now let's check if the index is correctly set when reading the saved file
path = '../stocks_data/AAPL_03783310_14593.csv'
df = pd.read_csv(path, index_col='datetime', parse_dates=True)  # Parse datetime and set as index
df['stock_exret']

datetime
2000-01-31    0.005019
2000-02-29    0.100519
2000-03-31    0.180142
2000-04-28   -0.091116
2000-05-31   -0.327922
                ...   
2023-08-31   -0.046954
2023-09-29   -0.092978
2023-10-31   -0.007270
2023-11-30    0.109320
2023-12-29    0.009283
Name: stock_exret, Length: 288, dtype: float64

In [49]:
cols = ['prc', 'target', 'log_diff', 'stock_exret', 'rf', 'adj_price', 'total_return']
df[cols]

,prc,target,log_diff,stock_exret,rf,adj_price,total_return
datetime,,,,,,,
2000-01-31,102.812500,1,0.000000,0.005019,0.0041,102.812500,0.000000
2000-02-29,103.750000,1,0.009077,0.100519,0.0043,103.750000,0.009119
2000-03-31,114.625000,1,0.099682,0.180142,0.0047,114.625000,0.104819
2000-04-28,135.812500,-1,0.169609,-0.091116,0.0046,135.812499,0.184842
2000-05-31,124.062500,-1,-0.090490,-0.327922,0.0050,124.062499,-0.086516
...,...,...,...,...,...,...,...
2023-08-31,196.449997,-1,0.012704,-0.046954,0.0045,25819.197721,0.012785
2023-09-29,187.869995,-1,-0.043381,-0.092978,0.0043,24723.080836,-0.042454
2023-10-31,171.210007,-1,-0.092859,-0.007270,0.0047,22530.680427,-0.088678


In [50]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [51]:
df

,date,ret_eom,permno,shrcd,exchcd,mspread,year,month,size_port,rf,...,qmj_prof,qmj_growth,qmj_safety,target,total_return,adj_price,log_price,log_diff,weight_attr,frac_diff
datetime,,,,,,,,,,,,,,,,,,,,,
2000-01-31,20000131,20000131,14593,11,3,0.016324,2000,1,B,0.0041,...,1.349406,1.051719,0.195078,1,0.000000,102.812500,4.632907,0.000000,0.005019,0.000000
2000-02-29,20000229,20000229,14593,11,3,0.013985,2000,2,B,0.0043,...,1.222134,0.694039,0.235076,1,0.009119,103.750000,4.641984,0.009077,0.100519,0.000000
2000-03-31,20000331,20000331,14593,11,3,0.016546,2000,3,B,0.0047,...,1.219227,0.687116,0.256001,1,0.104819,114.625000,4.741666,0.099682,0.180142,0.000000
2000-04-28,20000428,20000430,14593,11,3,0.027399,2000,4,B,0.0046,...,1.220738,0.686272,0.195822,-1,0.184842,135.812499,4.911275,0.169609,0.091116,0.000000
2000-05-31,20000531,20000531,14593,11,3,0.013961,2000,5,B,0.0050,...,0.879958,-0.700381,0.274371,-1,-0.086516,124.062499,4.820785,-0.090490,0.327922,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-31,20230831,20230831,14593,11,3,0.003065,2023,8,B,0.0045,...,1.666969,1.558169,0.401444,-1,0.012785,25819.197721,10.158874,0.012704,0.046954,1.580522
2023-09-29,20230929,20230930,14593,11,3,0.005090,2023,9,B,0.0043,...,1.666670,1.563366,0.368869,-1,-0.042454,24723.080836,10.115493,-0.043381,0.092978,1.512328
2023-10-31,20231031,20231031,14593,11,3,0.005148,2023,10,B,0.0047,...,1.665579,1.562672,0.406630,-1,-0.088678,22530.680427,10.022633,-0.092859,0.007270,1.431594


In [ ]:
df.index[0]

In [ ]:
# Define the ticker symbol and date range
ticker = 'AAPL'
start_date = df.index[0]
end_date = df.index[-1]

# Download the data
data = yf.download(ticker, start=start_date, end=end_date)
data['Close']


In [ ]:
fig, ax1 = plt.subplots(figsize=(12, 6))

# Plot data['Adj Close'] on the left y-axis
color = 'tab:blue'
ax1.set_xlabel('Date')
ax1.set_ylabel('Adj Close (Yahoo Finance)', color=color)
ax1.plot(data.index, data['Adj Close'], color=color)
ax1.tick_params(axis='y', labelcolor=color)

# Create a second y-axis for df['adj_price']
ax2 = ax1.twinx()
color = 'tab:red'
ax2.set_ylabel('Adj Price (Your Method)', color=color)
ax2.plot(df.index, df['adj_price'], color=color)
ax2.tick_params(axis='y', labelcolor=color)

plt.title('Comparison of Adjusted Prices of AAPL on Separate Y-Axes')
fig.tight_layout()
plt.show()


In [ ]:
fig, ax1 = plt.subplots(figsize=(12, 6))

# Plot data['Adj Close'] on the left y-axis
color = 'tab:blue'
ax1.set_xlabel('Date')
ax1.set_ylabel('Adj Close (Yahoo Finance)', color=color)
ax1.plot(df.index, df['prc'], color=color)
ax1.tick_params(axis='y', labelcolor=color)

# Create a second y-axis for df['adj_price']
ax2 = ax1.twinx()
color = 'tab:red'
ax2.set_ylabel('Adj Price (Your Method)', color=color)
ax2.plot(data.index, data['Close'], color=color)
ax2.tick_params(axis='y', labelcolor=color)

plt.title('Comparison of Adjusted Prices of AAPL on Separate Y-Axes')
fig.tight_layout()
plt.show()

In [ ]:
# load t1 from objects/t1.pkl
t1 = pd.read_pickle('../objects/t1.pkl')
t1

# Applying notes ch4 AFML on appl

In [ ]:
t1

In [37]:
def mpNumCoEvents(closeIdx, t1, molecule):
    '''
    Compute the number of concurrent events per bar.
    +molecule[0] is the date of the first event on which the weight will be computed
    +molecule[-1] is the date of the last event on which the weight will be computed
    Any event that starts before t1[molecule].max() impacts the count.
    '''
    #1) find events that span the period [molecule[0], molecule[-1]]
    t1 = t1.fillna(closeIdx[-1])  # unclosed events still must impact other weights
    t1 = t1[t1 >= molecule[0]]  # events that end at or after molecule[0]
    t1 = t1.loc[:t1[molecule].max()]  # events that start at or before t1[molecule].max()
    
    #2) count events spanning a bar
    iloc = closeIdx.searchsorted(np.array([t1.index[0], t1.max()]))
    count = pd.Series(0, index=closeIdx[iloc[0]:iloc[1] + 1])
    for tIn, tOut in t1.items():  # Changed from iteritems() to items()
        count.loc[tIn:tOut] += 1
    return count.loc[molecule[0]:t1[molecule].max()]

In [ ]:
df

In [ ]:
close = df['adj_price']
close

In [ ]:
numCoEvents = mpNumCoEvents(closeIdx = df.index, t1 = t1, molecule = t1.index)
numCoEvents = numCoEvents.loc[~numCoEvents.index.duplicated(keep="last")]
numCoEvents=numCoEvents.reindex(close.index).ﬁllna(0)
numCoEvents

In [41]:
def mpSampleTW(t1, numCoEvents, molecule):
    # Derive average uniqueness over the event's lifespan
    wght = pd.Series(index=molecule)
    for tIn, tOut in t1.loc[wght.index].items():  # Changed from .iteritems() to .items()
        wght.loc[tIn] = (1. / numCoEvents.loc[tIn:tOut]).mean()
    return wght

In [ ]:
mpSampleTW(t1 = t1, numCoEvents = numCoEvents, molecule = t1.index).mean()

In [43]:
# **SNIPPET 4.10** **DETERMINATION OF SAMPLE WEIGHT BY ABSOLUTE RETURN ATTRIBUTION**
def mpSampleW(t1, numCoEvents, close, molecule):
    # Derive sample weight by return attribution
    ret = np.log(close).diff()  # log-returns, so that they are additive
    wght = pd.Series(index=molecule)
    for tIn, tOut in t1.loc[wght.index].items():  # Changed from iteritems() to items()
        wght.loc[tIn] = (ret.loc[tIn:tOut] / numCoEvents.loc[tIn:tOut]).sum()
    return wght.abs()

In [ ]:
mpSampleW(t1 = t1, numCoEvents = numCoEvents, close = close, molecule = t1.index)

# Fracti diff

In [ ]:
# plot adj_price and stock_exret

import matplotlib.pyplot as plt
fig, ax1 = plt.subplots()
color = 'tab:red'
ax1.set_xlabel('time')
ax1.set_ylabel('price', color=color)
ax1.plot(df.index, df['frac_diff'], color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()
color = 'tab:blue'
ax2.set_ylabel('stock_exret', color=color)
ax2.plot(df.index, df['adj_price'], color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()
plt.show()

In [ ]:
# SNIPPET 5.3 THE NEW FIXED-WIDTH WINDOW FRACDIFF METHOD
def fracDiff_FFD(series, d, thres=1e-5):
    '''
    Constant width window (new solution)
    Note 1: thres determines the cut-off weight for the window
    Note 2: d can be any positive fractional, not necessarily bounded [0,1].
    '''
    #1) Compute weights for the longest series
    w = getWeights_FFD(d, thres)
    width = len(w) - 1
    #2) Apply weights to values
    df = {}
    for name in series.columns:
        # Corrected the fill method to 'ffill'
        seriesF, df_ = series[[name]].fillna(method='ffill').dropna(), pd.Series()
        for iloc1 in range(width, seriesF.shape[0]):
            loc0, loc1 = seriesF.index[iloc1-width], seriesF.index[iloc1]
            if not np.isfinite(series.loc[loc1, name]):
                continue  # exclude NAs
            df_[loc1] = np.dot(w.T, seriesF.loc[loc0:loc1])[0, 0]
        df[name] = df_.copy(deep=True)
    df = pd.concat(df, axis=1)
    return df


def getWeights_FFD(d, thres):
    # Appends while the last weight is above the threshold
    w = [1.]
    while abs(w[-1]) > thres:
        w_ = -w[-1] / len(w) * (d - len(w) + 1)
        w.append(w_)
    w = np.array(w[::-1]).reshape(-1, 1)
    return w

fracDiff_FFD(df[['adj_price']], d=0.5, thres=0.01)

In [ ]:
def plotMinFFD(df, col_name = 'adj_price'):
    from statsmodels.tsa.stattools import adfuller
    import matplotlib.pyplot as plt
    d_found = False
    
    out = pd.DataFrame(columns=['adfStat', 'pVal', 'lags', 'nObs', '95% conf', 'corr'])
    
    # Assuming 'Close' is the column name in df
    for d in np.linspace(0, 1, 21):
        df1 = np.log(df[[col_name]])# .resample('1D').last()  # Downcast to daily observations
        df2 = fracDiff_FFD(df1, d, thres=.01)
        corr = np.corrcoef(df1.loc[df2.index, col_name], df2[col_name])[0, 1]
        adf_result = adfuller(df2[col_name], maxlag=1, regression='c', autolag=None)
        out.loc[d] = list(adf_result[:4]) + [adf_result[4]['5%']] + [corr]  # With critical value
        print(f'Fractional differentiation order: {d}, ADF Statistic: {adf_result[0]}, Correlation: {corr}')
        if not d_found and adf_result[1] < 0.05:
            d_found = True
            d_req = d
            print(f'Fractional differentiation order found: {d_req}')

    # Plotting the results
    out[['adfStat', 'corr']].plot(secondary_y='adfStat')
    plt.axhline(out['95% conf'].mean(), linewidth=1, color='r', linestyle='dotted')
    plt.title('ADF Statistic and Correlation vs. Differentiation Order')
    plt.show()
    
    return out, d_req

# Assuming 'df' is the DataFrame in the local environment
result, d = plotMinFFD(df)

In [ ]:
import matplotlib.pyplot as plt
fig, ax1 = plt.subplots()
color = 'tab:red'
ax1.set_xlabel('time')
ax1.set_ylabel('price', color=color)
ax1.plot(df.index, df['adj_price'], color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()
color = 'tab:blue'
ax2.set_ylabel('Fractioned Diff', color=color)
col = 'log_price'
frac_diff = fracDiff_FFD(df[[col]], d=d, thres=0.01)
ax2.plot(frac_diff.index, frac_diff, color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()
plt.show()

In [ ]:
df[['log_price']]

In [ ]:
frac_diff

In [ ]:
df_ = df.copy()
df_['frac'] = frac_diff
df_['frac']=df_['frac'].fillna(0)
df_